# **Linear-nonlinear-Poisson model – LNPM**

The LNP model is a simplified model that has proven useful to describe the output of single neurons. The steps for  implementing it are described in the following schematic:  

<img src="LNP_flowchart.png">


This diagram also serves to outline the steps you'll need to take to make a simulation with the LNP model.

1). The stimulus. This one's easy. If you remember, the contrast values or the sequence of patterns that make up the stimuli we use are randomized by drawing random numbers. For the simulation, a sequence of random numbers suffices as stimulus. 

2). Under this model, a linear filter will determine (partly) the neuron's spiking output by filtering the signal. In very general terms, filtering means letting through what we want while letting out what we don't. A neuron wants signals to go through that are as similar as possible to its filter, while ignoring signals that differ from it. The neuron has no business dealing with those. In the simulation, to do the filtering, we should somehow compare the filter with the entire signal. The mathematical operation for this is a convolution, which will "slide" the filter along the signal making a point-to-point multiplication at each step. Similarities will produce larger positive values, whereas discrepancies will produce smaller or negative values. 

<img src="convolution.gif">


3). The result of convolving ("filtering") the signal with a linear filter is called the generator.

4). Once you have the generator, you will apply a **nonlinearity** to it in order to define the firing rate at each point.

5). One of the assumptions of the LNP model is that spiking is a Poisson process: spikes are stochastic and independent from each other, and the average rate (spikes per interval) is constant, although the probability of having a spike at any given time point is zero. For you to decide whether a given firing rate produces a spike or not, you multiply it by a time bin of your choice (the same for all) and compare it to a random number. If the random numbers is smaller, there's a spike; if it's not, there's no spike.

6). Now that you have the spikes, you should to be able to recover the filter (the STA) as you did earlier with the real cell data.

# Alternative


For this exercise, it is easier if you work with discrete time units. Practically this means dividing time into bins of equal size. Let's call this `dt`, start by assigning a reasonable number. This could be on the order of 1 ms, this way we can think of every bin as having one or zero spikes. You can set `dt` to 1 for this, thinking of a unit of time as 1 ms, or to 0.001 which would mean the unit of time would be seconds.

## Stimulus

Now we can start with creating the stimulus. First we need to decide how long our stimulus will be. You can think of this in terms of time (seconds, ms) or number of bins. We should have a long enough stimulus to be able to calculate statistics, but not too long so that it becomes difficult to calculate.

Our stimulus will be generated by choosing numbers from a normal distribution with a mean of `0` and a standard deviation of `1`.

Look for a numpy function that gives you normally distributed numbers. 

## Linear filter

The linear filter will decide what your neuron is "interested" in. For retinal cells, a filter extending to 600 ms before the spike is typically long enough to capture everything, with most of the interesting stuff happening between 0 and 200 ms before the spike. To give you an idea, it might look something like this.

<img src="linear_filter.png" width="350">

To have something like these bumps, you can use a Gaussian-like function:

$$
f(t) = a e^{-(t-b)^{2}/2c^{2}}
$$

where $a$ is the amplitude, $b$ is the center of the bump and $c$ is the width.

You can plot your filter, to see whether you have the shape you want.


## Convolution/Filtering

The operation we use to apply the filter to the stimulus is called convolution. Numpy has a function for this, again take a look at the parameters. If you would like to see some examples, check the online documentation for numpy.

The result of the this operation is called the generator signal, it represents the similarity of the stimulus to your filter, higher numbers mean more similar. The generator signal should have the same shape as your stimulus.

## Nonlinearity

In order to link the generator signal to the activation of the cell (i.e. instantaneous firing rate), we need to apply a nonlinear transformation. Don't let the name fool you, you can also have a nonlinearity that looks like a line.

Keep in mind that neurons cannot have a negative firing rate, so the output of the nonlinear transformation should be positive.

A basic nonlinear function would be a rectifying linear function; if generator is less than zero, zero spikes; otherwise the firing rate is the generator signal scaled by some number.

## Poisson process

In LNP, generation of spikes are modelled as a inhomogeneous (i.e. changing in time) Poisson process. We can generate spikes based on the instantaneous firing rate. For this we treat the firing rate at each time bin as the intensity of the Poisson process and the resulting number gives the number of spikes that happened in that time interval.

Check whether numpy has a function you can use for this purpose.

## Spikes!

If everything went well, you should have an array, with number of spikes at each time bin.

To do a sanity check, you can look at a few things:

- Your array should have the same shape as your stimulus.
- Based on your `dt` value, what should the maximum number of spikes per bin be, approximately?

If you have more or less spikes than expected, you can go back and try to change the nonlinearity or the filter.

How can you visualize the spikes? You can try plotting the number of spikes per bin with a line, or as dots. (Matplotlib documentation should help on the latter.)

We have used `eventplot` to visualize spike times before, you can think about trying to convert binned spikes into spike times, if you'd like to use that function.


On day 1, we learned about basic spike train statistics. You can apply those analyses on your spikes to explore.

You can experiment with different linear filters and nonlinear functions. How do they change the spike train statistics?

If you are looking for more still, you can try to recover the linear filter you put in by spike-triggered average. How can you do it with binned spikes?

How would you recover the nonlinearity?

How does the STA look like when you change the nonlinear function to be symmetrical with respect to the y axis?